In [1]:
import pandas as pd

In [2]:
movies = pd.read_csv("data/tmdb_5000_movies.csv")

In [3]:
movies.describe

<bound method NDFrame.describe of          budget                                             genres  \
0     237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1     300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2     245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3     250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4     260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
...         ...                                                ...   
4798     220000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4799       9000  [{"id": 35, "name": "Comedy"}, {"id": 10749, "...   
4800          0  [{"id": 35, "name": "Comedy"}, {"id": 18, "nam...   
4801          0                                                 []   
4802          0                [{"id": 99, "name": "Documentary"}]   

                                               homepage      id  \
0                           http://www.avatarmovie.com/   

In [4]:
import ast

movies["genre_names"] = movies["genres"].apply(
    lambda x: [d["name"] for d in ast.literal_eval(x)] if x != "[]" else []
)

movies["genre_names"]

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4798                        [Action, Crime, Thriller]
4799                                [Comedy, Romance]
4800               [Comedy, Drama, Romance, TV Movie]
4801                                               []
4802                                    [Documentary]
Name: genre_names, Length: 4803, dtype: object

In [5]:
movies = movies[['title','id', 'overview', 'genre_names', 'keywords']]
movies.dropna(inplace=True)

/tmp/ipykernel_91057/2310842046.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies.dropna(inplace=True)


In [6]:
movies['tags'] = movies['overview'] + " " + movies['genre_names'].apply(lambda x: " ".join(x))*4 + " " + movies['keywords'] +" "+movies['title']
movies.tags.to_csv('data/model_data.joblib')


/tmp/ipykernel_91057/349742125.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['tags'] = movies['overview'] + " " + movies['genre_names'].apply(lambda x: " ".join(x))*4 + " " + movies['keywords'] +" "+movies['title']


In [7]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
movies['tags'] = movies['tags'].apply(lambda x: " ".join([ps.stem(word) for word in x.split()]))


/tmp/ipykernel_91057/1468118738.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['tags'] = movies['tags'].apply(lambda x: " ".join([ps.stem(word) for word in x.split()]))


In [8]:
from sklearn.metrics.pairwise import cosine_similarity


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
vectors = vectorizer.fit_transform(movies['tags']).toarray()


In [10]:
similarity = cosine_similarity(vectors)

In [11]:
import joblib 
joblib.dump(similarity,"model/recommend_system.joblib")
joblib.dump(movies,"model/movies_data.joblib")

['model/movies_data.joblib']

In [14]:
def recommend(movie):
    movie_index = movies[movies['title'] == movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(
        list(enumerate(distances)), reverse=True, key=lambda x: x[1]
    )[1:6]
    return [movies.iloc[i[0]].title for i in movie_list]


In [16]:
recommend("Interstellar")
# Output: ['Interstellar', 'The Prestige', 'The Matrix', 'Memento', 'Shutter Island']


['Silent Running',
 'The Matrix Revolutions',
 'The Fifth Element',
 'Southland Tales',
 'The Astronaut Farmer']

In [69]:
def recommend(movie):
    movie_index = movies[movies['title'] == movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])
    
    # Filter only movies in the same main genre
    target_genre = movies.iloc[movie_index].genres.split()[0]
    recommendations = [
        movies.iloc[i[0]].title
        for i in movie_list
        if target_genre in movies.iloc[i[0]].genres
    ][:5]
    return recommendations


In [68]:
recommend("Cypher")

['City of Life and Death',
 'Inchon',
 'Windtalkers',
 'The Flowers of War',
 'Fort McCoy']

In [17]:
recommend("Cypher")

['Windtalkers',
 'Inchon',
 'The Flowers of War',
 'Letters from Iwo Jima',
 'Saving Private Ryan']

In [ ]:
recommend("Interstellar")

['Interstellar',
 'Silent Running',
 'Moonraker',
 'The Martian',
 '2001: A Space Odyssey']

In [19]:
recommend("Silent Running")

['Freakonomics',
 'Fish Tank',
 'Amour',
 "America's Sweethearts",
 'The Importance of Being Earnest']